In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [16]:
from policyengine_us import Simulation


situation = {
    "people": {
        "you": {
            "age": {"2025": 40, "2026": 40},
            "capital_gains_before_response": {"2025": 100000, "2026": 100000},
            "employment_income": {"2025": 200000, "2026": 200000},
        },
        "your partner": {"age": {"2025": 40, "2026": 40}},
        "your first dependent": {"age": {"2025": 10, "2026": 10}},
        "your second dependent": {"age": {"2025": 10, "2026": 10}},
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {"members": ["you", "your partner"]},
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2025": 1, "2026": 1},
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2025": 2, "2026": 2},
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ],
            "state_name": {"2025": "CA", "2026": "CA"},
        }
    },
    "axes": [[{"name": "real_estate_taxes", "count": 601, "min": 0, "max": 300000}]],
}

simulation = Simulation(
    situation=situation,
)

# Calculate for both 2025 and 2026
simulation = Simulation(situation=situation)

In [17]:
# Extract real estate tax values from the axes
real_estate_taxes = np.linspace(
    situation["axes"][0][0]["min"],
    situation["axes"][0][0]["max"],
    situation["axes"][0][0]["count"],
)

In [18]:
# Initialize a list to store results for both years
all_results = []

In [19]:
# Loop through 2025 and 2026
for year in [2025, 2026]:
    # Calculate household net income for the current year
    household_net_income = simulation.calculate("household_net_income", period=year)

    # Calculate the change in net income
    change_in_net_income = np.diff(household_net_income)

    # Calculate the change in real estate taxes
    change_in_real_estate_taxes = np.diff(real_estate_taxes)

    # Calculate the subsidy rate as a percentage
    subsidy_rate = (change_in_net_income / change_in_real_estate_taxes) * 100

    # Create a DataFrame for the current year
    results = pd.DataFrame(
        {
            "Year": year,  # Add a column for the year
            "Real Estate Taxes": real_estate_taxes[
                :-1
            ],  # Exclude the last value since diff reduces the size by 1
            "Change in Real Estate Taxes": change_in_real_estate_taxes,
            "Change in Net Income": change_in_net_income,
            "Subsidy Rate (%)": subsidy_rate,  # Subsidy rate as a percentage
        }
    )

    # Append the results to the list
    all_results.append(results)

In [20]:
# Combine results for both years into a single DataFrame
combined_results = pd.concat(all_results, ignore_index=True)

# Save the combined results to a CSV file
combined_results.to_csv("subsidy_rates.csv", index=False)